In [49]:
import pandas as pd
import os
import sys
import datetime
import urllib
import numpy as np
import cv2
import pymysql

from pathlib import Path
from enum import Enum
from io import BytesIO, StringIO
from typing import Union

from detection import upload_to_gcs
from detection import generate_download_signed_url_v4
from detection import get_similar_products_uri
from detection import query_product

In [50]:
project_id = 'abstract-veld-289612'
location = 'asia-east1'
product_set_id = 'PAIR-Filter'
bucket_name = 'ftmle'
product_category= 'homegoods-v2'

In [51]:
def search_product(bucket_name, blob_name):
    serving_url = generate_download_signed_url_v4(bucket_name, blob_name)
    res = get_similar_products_uri(project_id, location, product_set_id, product_category,
        image_uri=serving_url, filter='')
    return res, serving_url

In [91]:
#Setup Connection to mysql database
conn = pymysql.connect(
    host='35.221.181.94',
    port=int(3306),
    user="mkhoa",
    passwd='NTMK261194@dng',
    db="project",
    charset='utf8mb4')

cur = conn.cursor()

def query_room():
    '''
    
    '''
    query = '''
    SELECT a.id as 'room_id', a.subtitle as 'room_type', b.bucket_path
    FROM project.DesignsDetail a
    LEFT JOIN Files b ON a.id = b.id
    WHERE b.bucket_path like '%/Design/%'
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY INSERT:', err)
    result = cur.fetchall()
    return result

In [92]:
room = pd.DataFrame(query_room(), columns=['room_id', 'room_type', 'bucket_path'])

In [93]:
test = room[:3]

In [94]:
test.values

array([[5325, 'Contemporary Living Room, Omaha',
        'Images/Design/5325-home-design.jpg'],
       [5326, 'Transitional Living Room',
        'Images/Design/5326-home-design.jpg'],
       [5327, 'Contemporary Living Room, Los Angeles',
        'Images/Design/5327-home-design.jpg']], dtype=object)

In [95]:
for idx, row in test.iterrows():
    blob = row['bucket_path']
    res, serving_uri = search_product(bucket_name, blob)
    print(res)

[bounding_poly {
  normalized_vertices {
    x: 0.4940434396266937
    y: 0.6450653076171875
  }
  normalized_vertices {
    x: 0.9383861422538757
    y: 0.6450653076171875
  }
  normalized_vertices {
    x: 0.9383861422538757
    y: 0.9890355467796326
  }
  normalized_vertices {
    x: 0.4940434396266937
    y: 0.9890355467796326
  }
}
results {
  product {
    name: "projects/abstract-veld-289612/locations/asia-east1/products/137821"
    display_name: "Carlyle Lounge Chair  Saloon Light Gray Leather"
    product_category: "homegoods-v2"
    product_labels {
      key: "\"category"
      value: "Armchairs and Accent Chairs\""
    }
  }
  score: 0.42131319642066956
  image: "projects/abstract-veld-289612/locations/asia-east1/products/137821/referenceImages/54d81555c2c463a83f0136b014fad5e6"
}
results {
  product {
    name: "projects/abstract-veld-289612/locations/asia-east1/products/63888"
    display_name: "Hunter Solid Mango Wood End Table  Umber Brown Stain"
    product_category: "h

In [96]:
for i in range(len(res)):
    for j in range(len(res[i].results)):
        if res[0].results[1].score > 0.55
        res[i].results[j]

SyntaxError: invalid syntax (<ipython-input-96-bfd93595efa3>, line 3)

In [136]:
result = []
for i in range(len(res)):    
    for j in range(len(res[i].results)):
        d = {}
        d['object'] = i
        d['idx'] = int(res[i].results[j].product.name.split('/', -1)[-1])
        d['name'] = res[i].results[j].product.display_name
        d['score'] = res[i].results[j].score
        result.append(d)

In [137]:
df = pd.DataFrame(result)

In [131]:
def query_product(id):
    '''
    
    '''
    query = f'''
    SELECT id, website, product, image, url, b.group_category 
    FROM project.ProductHeader a
    LEFT JOIN project.category b ON a.category = b.Category
    WHERE id = '{id}'
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY INSERT:', err)
    result = cur.fetchone()
    return result

In [135]:
query = pd.DataFrame(df['idx'].apply(query_product).tolist(), columns=['idx', 'Website', 'Product Name', 'Image', 'URL', 'Category'])

In [142]:
result = df.merge(query)

In [148]:
for i in result['Category'].unique():
    print(i)

Bench
Object
Pillows
Table
Chair
Bed
Sofa
Lighting
